In [1]:
import math
import numpy as np
import pandas as pd
from numpy import arange
from matplotlib import pyplot
from pandas import read_csv
from pandas.plotting import scatter_matrix

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor

#from sklearn.externals.joblib import dump
#from sklearn.externals.joblib import load
#from sklearn.feature_selection import RFE

from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

# Create one random seed number for reproducible results
seedNum = 888

# Libraries for neural network
import tensorflow
import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

In [2]:
outlier_flag = True      # Will threshold the outliers to outlier_max_value value
outlier_max_value = 9500
scale_flag = False
scale_cols=['n_tokens_title','average_token_length']

In [3]:
# Display all columns in pandas
pd.set_option('display.max_columns', None)

### Reading data

In [4]:
inputFile = '../data/output/OnlineNewsPopularity_featureEngg_RecEli.csv' #Kannan stiched this file
df = read_csv(inputFile)
df.columns = df.columns.str.replace(' ','')
print(df.shape)
df.head()

(7795, 139)


,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,num_keywords,data_channel_is_lifestyle,data_channel_is_entertainment,data_channel_is_bus,data_channel_is_socmed,data_channel_is_tech,data_channel_is_world,is__Monday,is__Tuesday,is__Wednesday,is__Thursday,is__Friday,is__Saturday,is__Sunday,is_Weekend,kw_avg_avg,kw_min_avg,kw_max_avg,kw_avg_avg_no_clip,kw_min_avg_no_clip,cluster_0,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7,cluster_8,cluster_9,polarity_title,subjectivity_title,polarity_content,subjectivity_content,keywords_pca_0,keywords_pca_1,keywords_pca_2,keywords_pca_3,keywords_pca_4,keywords_pca_5,keywords_pca_6,keywords_pca_7,keywords_pca_8,keywords_pca_9,content_pca_0,content_pca_1,content_pca_2,content_pca_3,content_pca_4,content_pca_5,content_pca_6,content_pca_7,content_pca_8,content_pca_9,NER_list_pca_0,NER_list_pca_1,NER_list_pca_2,NER_list_pca_3,NER_list_pca_4,NER_list_pca_5,NER_list_pca_6,NER_list_pca_7,NER_list_pca_8,NER_list_pca_9,NER_GPE,NER_DATE,NER_CARDINAL,NER_NORP,NER_PERSON,NER_TIME,NER_ORG,NER_WORK_OF_ART,NER_QUANTITY,NER_EVENT,NER_ORDINAL,NER_MONEY,NER_FAC,NER_PRODUCT,NER_LAW,NER_PERCENT,NER_LOC,NER_LANGUAGE,LDA_T0,LDA_T1,LDA_T2,LDA_T3,LDA_T4,NMF_T0,NMF_T1,NMF_T2,NMF_T3,NMF_T4,LSI_T0,LSI_T1,LSI_T2,LSI_T3,LSI_T4,LDA_C0,LDA_C1,LDA_C2,LDA_C3,LDA_C4,NMF_C0,NMF_C1,NMF_C2,NMF_C3,NMF_C4,LSI_C0,LSI_C1,LSI_C2,LSI_C3,LSI_C4,LDA_K0,LDA_K1,LDA_K2,LDA_K3,LDA_K4,NMF_K0,NMF_K1,NMF_K2,NMF_K3,NMF_K4,LSI_K0,LSI_K1,LSI_K2,LSI_K3,LSI_K4,Id,shares
0,9,967,0.422958,0.467425,0.723716,7,6,0,0,4.172699,7,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1696,879,2224,2091,879,0,0,0,0,0,0,0,0,0,1,0.000,0.00,0.013542,0.390972,13.136436,-0.564639,-0.715556,-0.365038,0.279909,-0.919380,0.012948,-0.423773,-0.529433,-0.417151,20.136116,20.801281,-8.014767,-13.831778,16.490318,-9.784305,2.202778,8.778829,7.295693,10.242371,71.769478,-0.305974,2.390489,-2.477139,0.053204,-0.480504,0.319525,0.988350,-0.227759,1.380270,37,16,5,10,9,2,7,1,1,0,0,0,0,0,0,0,0,0,0.033334,0.033334,0.033334,0.866665,0.033334,0.000000,0.000091,0.000629,0.014073,0.007566,0.007664,0.009979,0.018342,0.063746,0.056000,0.000506,0.044953,0.000507,0.032436,0.921598,0.000000,0.000000,0.717183,0.000000,0.080817,9.623200,-6.378665,-4.689331,-8.624384,1.097369,0.050000,0.050000,0.799682,0.050318,0.050000,0.174287,0.000000,0.000000,0.000245,0.00000,1.337092,-0.312890,-0.075333,0.014327,-0.026230,1,1100
1,14,290,0.606897,0.472414,0.596591,6,4,1,0,4.200000,9,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1986,575,2971,2787,575,0,0,0,1,0,0,0,0,0,0,0.000,0.00,0.056136,0.518788,6.786283,0.691627,1.088876,0.853832,-0.138643,0.458027,0.407789,0.523944,0.875752,-0.142530,13.016478,-12.000452,11.389012,-6.300879,-1.136105,8.273910,-4.287179,3.648927,3.724643,2.798712,21.325382,-21.734667,-0.771340,-1.550308,1.541853,-0.771978,0.003611,-1.077202,1.362484,0.993559,1,3,0,0,2,0,15,1,0,1,0,0,0,0,0,0,0,0,0.033416,0.033334,0.033334,0.033334,0.866582,0.000000,0.261545,0.000000,0.000000,0.000000,0.201663,1.093149,-0.088902,-0.083704,-0.005675,0.001565,0.163156,0.001580,0.792166,0.041534,0.057204,0.324944,0.004722,0.000297,0.000000,4.020826,4.933150,-2.165117,-0.008496,0.364554,0.028572,0.885713,0.028572,0.028572,0.028572,0.000000,0.000000,0.141952,0.000000,0.16111,0.108039,0.157404,1.083515,-0.085414,0.859248,2,1100
2,10,392,0.591837,0.545918,0.698276,9,1,1,0,4.545918,6,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1829,1000,2456,2402,1000,0,1,0,0,0,0,0,0,0,0,-0.075,0.05,0.003831,0.348095,0.567927,0.097915,-0.608448,-1.120011,0.259292,1.026721,0.301655,0.340355,0.134292,0.656617,13.522844,22.819269,-3.552361,12.605568,-5.023582,-6.502191,6.504125,-0.497396,-4.285446,-5.146639,63.302799,-10.992071,-1.079108,5.448328,1.981348,-1.181570,0.729349,2.825014,0.221779,-1.192644,2,11,3,4,17,0,12,0,0,0,1,1,0,0,0,0,0,0,0.033334,0.033334,0.199798,0.033334,0.700199,0.000652,0.000000,0.003427,0.001326,0.003812,0.007070,0.002420,0.01680

In [5]:
# Dropping useless attributes
df.drop('url', axis=1, inplace=True)
df.drop('Id', axis=1, inplace=True)

KeyError: "['url'] not found in axis"

In [ ]:
# variables for total columns, total input attributes, target column index and renaming target variable
totCol = len(df.columns)
totAttr = totCol-1
targetCol = totCol #last column
df.rename(columns={'shares': 'targetVar'}, inplace=True)

In [ ]:
# Threshold the value > outlier_max_value to value outlier_max_value itself in column shares or targetVar
if outlier_flag:
    print(df[(df['targetVar'] == outlier_max_value)].shape)
    print(df[(df['targetVar'] > outlier_max_value)].shape)
    df['targetVar'] = df['targetVar'].clip(upper = outlier_max_value)
    print(df[(df['targetVar'] == outlier_max_value)].shape)
    print("shape is: ",df.shape)

In [ ]:
#70:30 train test split
if targetCol == totCol :
    x_df = df.iloc[:,0:totAttr]
    y_df = df.iloc[:,totAttr]
else :
    x_df = df.iloc[:,1:totCol]
    y_df = df.iloc[:,0]

validation_size = 0.30
x_train, x_test, y_train, y_test = train_test_split(x_df, y_df, test_size=validation_size, random_state=seedNum)
print("x_df.shape: {} y_df.shape: {}".format(x_df.shape, y_df.shape))
print("x_train.shape: {} y_train.shape: {}".format(x_train.shape, y_train.shape))
print("x_test.shape: {} y_test.shape: {}".format(x_test.shape, y_test.shape))

In [ ]:
# visualization configs
dispCol = 3
if totAttr % dispCol == 0 :
    dispRow = totAttr // dispCol
else :
    dispRow = (totAttr // dispCol) + 1
    
# Set figure width to 16 and height to 12 (4:3 aspect ratio)
fig_size = pyplot.rcParams["figure.figsize"]
fig_size[0] = 16
fig_size[1] = 12
pyplot.rcParams["figure.figsize"] = fig_size

In [ ]:
#kannan: This is decided from feature engineering / feature selection
lowAttributes=[]

In [ ]:
# Removing the unselected attributes from the training and validation dataframes
#xy_train <- xy_train[, !(names(xy_train) %in% lowAttributes)]
#xy_test <- xy_test[, !(names(xy_test) %in% lowAttributes)]
xy_newdf = df.drop(lowAttributes, axis=1)

totCol = len(xy_newdf.columns)
totAttr = totCol-1
targetCol = totCol
#array = xy_newdf.values

if targetCol == totCol :
    #x_newdf = array[:,0:totAttr]
    #y_newdf = array[:,totAttr]
    x_newdf = xy_newdf.iloc[:,0:totAttr]
    y_newdf = xy_newdf.iloc[:,totAttr]

else :
    #x_newdf = array[:,1:totCol]
    #y_newdf = array[:,0]
    x_newdf = xy_newdf.iloc[:,1:totCol]
    y_newdf = xy_newdf.iloc[:,0]

validation_size = 0.30
x_train, x_test, y_train, y_test = train_test_split(x_newdf, y_newdf, test_size=validation_size, random_state=seedNum)
print("x_newdf.shape: {} y_newdf.shape: {}".format(x_newdf.shape, y_newdf.shape))
print("x_train.shape: {} y_train.shape: {}".format(x_train.shape, y_train.shape))
print("x_test.shape: {} y_test.shape: {}".format(x_test.shape, y_test.shape))

In [ ]:
x_train.head()

In [ ]:
x_test.head()

In [ ]:
#scale_cols=['n_tokens_title','average_token_length']

In [ ]:
if scale_flag:
    
    def scale_trn(df, columns):
        for col in columns:
            df[col] = (df[col] - df[col].mean())/df[col].std(ddof=0) 
        return df

    def scale_tst(df, df_trn_tmp, columns):
        for col in columns:
            df[col] = (df[col] - df_trn_tmp[col].mean())/df_trn_tmp[col].std(ddof=0) 
        return df
    
    x_train = scale_trn(x_train.copy(), scale_cols)
    x_test  = scale_tst(x_test.copy(), x_train.copy(), scale_cols)

In [ ]:
x_train.head()

In [ ]:
if isinstance(x_train, pd.DataFrame):
    x_train = x_train.values
    y_train = y_train.values
    print("train data change from DataFrame to Numpy ")
    
if isinstance(x_test, pd.DataFrame):
    x_test = x_test.values
    y_test = y_test.values
    print("test data change from DataFrame to Numpy ")

## Traditional ML Models Evaluation

In [ ]:
# Comparing various types of regression # K-fold cross validation
num_folds = 10
scoring = 'neg_mean_squared_error'
#Other scoring options: https://scikit-learn.org/stable/modules/model_evaluation.html
models = []

# LINEAR ALGORITHMS
# -----------------
# Linear Regression
models.append(('LR', LinearRegression()))
# Ridge Regression
models.append(('RR', Ridge(random_state=seedNum)))
# Lasso Regression
models.append(('LASSO', Lasso(random_state=seedNum)))
# Elastic Net
models.append(('EN', ElasticNet(random_state=seedNum)))

# NON-LINEAR ALGORITHMS
# ---------------------
# Classification and Regression Trees
models.append(('CART', DecisionTreeRegressor(random_state=seedNum)))
# K Nearest Neighbor
models.append(('KNN', KNeighborsRegressor()))
# Suppor Vector machine
models.append(('SVM', SVR()))

# ENSEMBLE ALGORITHMS
# -------------------
# Extra Trees
models.append(('ETree', ExtraTreesRegressor(random_state=seedNum)))
# Random Forest
models.append(('RF', RandomForestRegressor(random_state=seedNum)))
# Stochastic Gradient Boosting
models.append(('GBM', GradientBoostingRegressor(random_state=seedNum)))
# Adaptive Boosting
models.append(('ABM', AdaBoostRegressor(random_state=seedNum)))

results = []
names = []
metrics1= [] #RMSE_Training
metrics2= [] #RMSE_Testing

## Adding DL Model

In [ ]:
def baseline_model(init_mode='uniform', neurons=5, learn_rate=0.001, momentum=0.2, act_function='relu'):
        
    model = Sequential()
    
    model.add(BatchNormalization())
    model.add(Dense(neurons, input_dim=17, kernel_initializer=init_mode, activation=act_function))
    model.add(Dense(6,  kernel_initializer=init_mode, activation=act_function))
    model.add(Dense(1,  kernel_initializer=init_mode))
    
    optimizer = tensorflow.keras.optimizers.RMSprop(lr=learn_rate, momentum=momentum)
    model.compile(loss='mean_squared_error', optimizer=optimizer, )
    
    return model

estimator = KerasRegressor(build_fn=baseline_model, epochs=50, batch_size=10, verbose=0)

In [ ]:
models.append(('DL', estimator))

## Training

In [ ]:
for name, model in models:
    startTimeModule = datetime.now()
    kfold = KFold(n_splits=num_folds, random_state=seedNum)
    cv_results = cross_val_score(model, x_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    rmse=math.sqrt(abs(cv_results.mean()))
    metrics1.append(rmse)
    print("\n%s: rmse:%f" % (name, rmse))
    print ('Model training time:',(datetime.now() - startTimeModule))

## Testing

In [ ]:
for name, model in models:
    model.fit(x_train, y_train)
    predictions=model.predict(x_test)
    rmse=math.sqrt(mean_squared_error(y_test, predictions))
    metrics2.append(rmse)
    print("\n%s: rmse:%f" % (name, rmse))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


labels = names

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

metrics1=[math.floor(item) for item in metrics1]
metrics2=[math.floor(item) for item in metrics2]

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, metrics1, width, label='TrainRMSE')
rects2 = ax.bar(x + width/2, metrics2, width, label='TestRMSE')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('RMSE')
ax.set_title('RMSE by Regression Algorithms')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()